In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing, model_selection
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import json
import random

### Data Preparation

In [2]:
def get_dataset(directory):
    data = pd.DataFrame(columns=['data', 'label'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                temp_label = filename.split('.')[0].split('_')[0]
                if 'a' in temp_label:
                    label ='alcoholic'
                else:
                    label = 'control'

                temp_data = pd.DataFrame(columns=['data', 'label'], index=[0])

                rwb = np.load(rel_path)
                rwb.astype(np.float64).reshape(-1,1)
                # print(rwb)
                # with open(rel_path, 'r') as file:
                    
                #     rwb = list(csv.reader(file, delimiter=","))[0]
                #     # scaler = preprocessing.MinMaxScaler()
                #     rwb = np.asarray(rwb).astype(np.float64).reshape(-1,1)
                #     # print(rwb)
                                
                temp_data['data'][0] = rwb
                temp_data['label'] = label
                
                # decomp = np.arange(0, 366)
                # plt.plot(decomp, df_data)
                # plt.xlabel('Dimension Number')
                # plt.ylabel('Wavelet Bispectrum Energy')
                # plt.show()
                data = pd.concat([data, temp_data], ignore_index=True)
    label_map = {"alcoholic": 1, "control": 0}
    data['label_map'] = data['label'].map(label_map)
    # print(data)      
    return data

In [3]:
def get_batch(path):
    # loading extracted feature & label
    x = get_dataset(path)

    scaler = preprocessing.MinMaxScaler()

    series_list = [
        i for i in x["data"]
    ]

    # series_list = series_list.reshape(-1, 366, 1)

    labels_list = [i for i in x["label_map"]]
        
    # y = keras.utils.to_categorical(y[0])
    dataset = tf.data.Dataset.from_tensor_slices((series_list,labels_list))
    dataset = dataset.shuffle(len(labels_list)).batch(32)

    return dataset

In [26]:
test = get_batch('../Feature/Bispectrum/Train/smni_cmi_train_bispectrum_256')
print([i[0].shape[1] for i in test][0])

915


In [ ]:
del test

### Exploratory Data Analysis

In [27]:
# Load the CSV file into a DataFrame
signal = np.load('../smni_cmi_test_bispectrum_64\co2a0000364\co2a0000364_48_bispectrum.npy')


# Extract the signal values from the DataFrame

# Create a time axis for the signal
t = range(len(signal))

fig, ax = plt.subplots(figsize=(10, 5))
# Plot the signal
ax.plot(t, signal)
ax.set_xlabel('Time (samples)')
ax.set_ylabel('Signal amplitude')
ax.set_title('Signal plot')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '../smni_cmi_test_bispectrum_64\\co2a0000364\\co2a0000364_48_bispectrum.npy'

### Model Definition

In [4]:
def create_model(feature):
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(feature,)))
    model.add(layers.Reshape((feature, 1)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

### Define Callbacks

In [5]:
def myCallbacks(log_dir):
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='acc',
    patience=50,
    mode='max')
    model_path = os.path.join(log_dir,'best_model.h5')
    mc = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_acc', mode='max', verbose=1, save_best_only=True)
    return [tensorboard_callback, early_stopping, mc]
    

### Main Program

In [6]:
lags = [256, 128, 64, 32, 16, 8, 4, 2]
folds = ['train_1', 'test_1', 'epoch_1', 'train_2', 'test_2', 'epoch_2']
time_measured = ['Wall_Time_1', 'CPU_Time_1', 'Wall_Time_2', 'CPU_Time_2']
epochs = 2000
log_dirs = ['logs1/Bispectrum_ANN', 'logs2/Bispectrum_ANN', 'logs3/Bispectrum_ANN', 'logs4/Bispectrum_ANN', 'logs5/Bispectrum_ANN']
train_dir = '../Feature/Bispectrum/Train/smni_cmi_train_bispectrum'
test_dir = '../Feature/Bispectrum/Test/smni_cmi_test_bispectrum'



In [10]:
for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured)
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch(train_temp_dir)
            test_ds = get_batch(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model([i[0].shape[1] for i in train][0])
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path))

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))


Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_48 (Reshape)        (None, 915, 1)            0         
                                                                 
 flatten_48 (Flatten)        (None, 915)               0         
                                                                 
 dense_144 (Dense)           (None, 512)               468992    
                                                                 
 dropout_96 (Dropout)        (None, 512)               0         
                                                                 
 dense_145 (Dense)           (None, 256)               131328    
                                                                 
 dropout_97 (Dropout)        (None, 256)               0         
                                                                 
 dense_146 (Dense)           (None, 1)               

In [2]:
! tensorboard --logdir=RWB:logs/RWB/,RWB_ANN:logs/RWB_ANN/,Bispectrum:logs/Bispectrum/,Bispectrum_ANN:logs/Bispectrum_ANN/ --port=6006

^C


In [3]:
tensorboard --logdir=RWB:logs/RWB/,RWB_ANN:logs/RWB_ANN/,Bispectrum:logs/Bispectrum/,Bispectrum_ANN:logs/Bispectrum_ANN/ --port=8000

^C


In [ ]:
! cd